<a href="https://colab.research.google.com/github/ymiftah/operations_research/blob/master/inheritance.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q pyomo
!apt-get install -qq coinor-cbc

     |████████████████████████████████| 2.3MB 2.9MB/s 
     |████████████████████████████████| 51kB 7.0MB/s 
     |████████████████████████████████| 256kB 54.4MB/s 
     |████████████████████████████████| 163kB 53.1MB/s 
Selecting previously unselected package coinor-libcoinutils3v5.
(Reading database ... 132681 files and directories currently installed.)
Preparing to unpack .../0-coinor-libcoinutils3v5_2.10.14+repack1-1_amd64.deb ...
Unpacking coinor-libcoinutils3v5 (2.10.14+repack1-1) ...
Selecting previously unselected package coinor-libosi1v5.
Preparing to unpack .../1-coinor-libosi1v5_0.107.9+repack1-1_amd64.deb ...
Unpacking coinor-libosi1v5 (0.107.9+repack1-1) ...
Selecting previously unselected package coinor-libclp1.
Preparing to unpack .../2-coinor-libclp1_1.16.11+repack1-1_amd64.deb ...
Unpacking coinor-libclp1 (1.16.11+repack1-1) ...
Selecting previously unselected package coinor-libcgl1.
Preparing to unpack .../3-coinor-libcgl1_0.59.10+repack1-1_amd64.deb ...
Unpacking coi

In [0]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pprint import pprint

# Splitting an inheritance

## Problem Statement
A rich aristocrat passes away, leaving the following legacy

|         | Empty Containers   |
|---------|-------------------:|
|A Caillebotte picture   | 25000|
|a bust of Diocletian  | 5000|
|A Yuan dynasty vase    | 20000|
|A 911 Porsche  | 40000|
|Three Diamonds  | 12000 each|
|A Louis XV Sofa  | 3000|
|Two race dogs  | 6000|
|A Sculpture dated 200 A.D.  | 10000|
|A Sailing Boat  | 15000|
|A Harley Davidson bike  | 10000|
|A precious piece of furniture  | 13000|

The will requires that the inheritance be split between the two sons as equally as possible

**Find the splitting minimizing the difference between the values ot the two parts**

In [0]:
from pyomo.environ import *

# create a model
model = ConcreteModel()

objects  =   ['Picture', 'Bust', 'Vase', 'Porsche',
              'Diamond_1', 'Diamond_2', 'Diamond_3',
              'Sofa', 'Dogs', 'Sculpture', 'Boat',
              'Bike', 'Furniture']
values       = dict(zip(objects, [25000, 5000, 20000, 40000,
                                12000, 12000, 12000,
                                3000, 6000, 10000, 15000,
                                10000, 13000
                                ]))

# Sets
model.objects = Set(initialize=objects)

# declare decision variables
model.to_a = Var(model.objects, domain=Binary)
model.diff = Var(domain=NonNegativeReals)

# declare objective
model.obj = Objective(expr = model.diff,
                       sense=minimize)

#declare constraints
def cstr_diff_1(m, i):
    return (sum(values[i] * (m.to_a[i] - (1-m.to_a[i]))
               for i in m.objects) <= m.diff)

def cstr_diff_2(m, i):
    return (sum(values[i] * (- m.to_a[i] + (1-m.to_a[i]))
               for i in m.objects) <= m.diff)


model.diff_1 = Constraint(model.objects, rule=cstr_diff_1)
model.diff_2 =  Constraint(model.objects, rule=cstr_diff_2)

In [9]:
# Solve the problem
SolverFactory('cbc', executable='/usr/bin/cbc').solve(model)

print('Difference = ', model.obj())

Cost =  1000.0


## Results

Let us visualize in a table the splitting

In [21]:
splitting = model.to_a.extract_values()

df = pd.DataFrame(splitting.values(),
                  columns=['Brother A'],
                  index=splitting.keys(),
                  dtype='int')
df['Brother B'] = 1-df['Brother A']
print(df)

           Brother A  Brother B
Picture            0          1
Bust               1          0
Vase               0          1
Porsche            1          0
Diamond_1          0          1
Diamond_2          0          1
Diamond_3          1          0
Sofa               0          1
Dogs               0          1
Sculpture          1          0
Boat               1          0
Bike               1          0
Furniture          0          1


The difference of values between the two brothers is only 1000$

In [22]:
print('Difference = ', model.obj())

Difference =  1000.0
